# 10.5 多头注意力
- **目录**
  - 10.5.1 多头注意力模型
  - 10.5.2 多头注意力实现


在实践中，当给定相同的查询、键和值的集合时，
我们希望模型可以**基于相同的注意力机制学习到不同的行为**，
然后将不同的行为作为知识组合起来，
**捕获序列内各种范围的依赖关系**
（例如，短距离依赖和长距离依赖关系）。
因此，允许**注意力机制组合使用查询、键和值的不同
子空间表示（representation subspaces）可能是有益的。**

为此，与其只使用单独一个注意力池化，
我们可以用独立学习得到的$h$组不同的
**线性投影（linear projections）**来变换查询、键和值。
然后，这$h$组变换后的查询、键和值将并行地送到注意力池化中。
最后，将这$h$个注意力池化的输出拼接在一起，
并且通过另一个可以学习的线性投影进行变换，
以产生最终输出。
这种设计被称为**多头注意力（multihead attention）**。
对于$h$个注意力池化输出，每一个注意力池化都被称作一个**头（head）**。
图10.5.1
展示了使用全连接层来实现可学习的线性变换的多头注意力。

<center><img src='../img/multi-head-attention.svg'/></center>
<center>图10.5.1 多头注意力：多个头连结然后线性变换</center>

- **要点：**
  - **多头注意力**：实践中，我们希望模型可以基于相同的注意力机制**学习到不同的行为**，然后将不同的行为作为知识**组合**起来，捕获序列内**各种范围的依赖关系**。为此，我们可以使用多头注意力，它允许注意力机制组合使用查询、键和值的**不同子空间表示**。
  - **子空间表示**：在多头注意力中，我们通过独立学习得到的$h$组不同的线性投影来变换查询、键和值。然后，这$h$组变换后的查询、键和值**并行**地送到注意力池化中。
  - **多头**：在多头注意力中，对于$h$个注意力池化输出，每一个注意力池化都被称作一个"头"。
  - **多头注意力的实现**：多头注意力的实现包括：使用全连接层来实现**可学习的线性变换**，将$h$个注意力池化的输出拼接在一起，并且通过另一个**可以学习的线性投影**进行变换，以产生最终输出。
  - **线性投影**是一种线性变换，它将数据从**一个向量空间映射到另一个向量空间**。在深度学习中，这种线性投影通常由一个全连接层（或线性层）执行，**没有激活函数**。
    - 线性投影的主要作用是改变数据的维度或表示形式，以便更好地进行后续的计算和处理。

## 10.5.1 多头注意力模型

- 首先用数学语言将多头注意力模型形式化地描述出来。
- 给定查询$\mathbf{q} \in \mathbb{R}^{d_q}$、
键$\mathbf{k} \in \mathbb{R}^{d_k}$和
值$\mathbf{v} \in \mathbb{R}^{d_v}$，
每个注意力头$\mathbf{h}_i$（$i = 1, \ldots, h$）的计算方法为：
$$\mathbf{h}_i = f(\mathbf W_i^{(q)}\mathbf q, \mathbf W_i^{(k)}\mathbf k,\mathbf W_i^{(v)}\mathbf v) \in \mathbb R^{p_v} \tag{10.5.1}$$
  其中，可学习的参数包括：
  - $\mathbf W_i^{(q)}\in\mathbb R^{p_q\times d_q}$、
  - $\mathbf W_i^{(k)}\in\mathbb R^{p_k\times d_k}$和
  - $\mathbf W_i^{(v)}\in\mathbb R^{p_v\times d_v}$，
  - 以及代表注意力池化的函数$f$（实际上$f$中所包含的权重矩阵可以被学习）。
- $f$可以是 10.3节中加性注意力和缩放点积注意力。
- 多头注意力的输出需要经过另一个线性转换，
它对应着$h$个头连结后的结果，因此其可学习参数是
$\mathbf W_o\in\mathbb R^{p_o\times h p_v}$：
$$\mathbf W_o \begin{bmatrix}\mathbf h_1\\\vdots\\\mathbf h_h\end{bmatrix} \in \mathbb{R}^{p_o}\tag{10.5.2}$$
- 基于这种设计，每个头都可能会关注输入的不同部分，可以表示比简单加权平均值更复杂的函数。

In [18]:
%matplotlib inline
import math
import torch
from torch import nn
from d2l import torch as d2l

## 10.5.2 多头注意力实现

- 在实现过程中，我们选择缩放点积注意力作为每一个注意力头。
- 为了避免计算代价和参数代价的大幅增长，
我们设定$p_q = p_k = p_v = p_o / h$。
- 值得注意的是，如果我们将查询、键和值的线性变换的输出数量设置为
$p_q h = p_k h = p_v h = p_o$，
则可以并行计算$h$个头。
- 在下面的实现中，$p_o$是通过参数`num_hiddens`指定的。


In [19]:
#@save
class MultiHeadAttention(nn.Module):
    """多头注意力"""
    '''
     key_size, query_size, value_size, num_hiddens都是100；
     num_heads：5
    '''
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias) # (100, 100)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias) # (100, 100)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias) # (100, 100)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias) # 100, 100)

    '''
    quereies: (2, 4, 100)
    keys: (2, 6, 100)
    values:(2, 6, 100)
    
    查询有4个，键-值对有6个。
    有5个注意力头。
    '''
    def forward(self, queries, keys, values, valid_lens):
        # queries，keys，values的形状:
        # (batch_size，查询或者“键－值”对的个数，num_hiddens)
        # valid_lens　的形状:
        # (batch_size，)或(batch_size，查询的个数)
        # 经过变换后，输出的queries，keys，values　的形状:
        # (batch_size*num_heads，查询或者“键－值”对的个数，
        # num_hiddens/num_heads)
        '''
        queries的形状会变形为：(2, 4, 100)->(2*5, 4, 100/5)=(10, 4, 20)
        keys的形状会变形为：(2, 6, 100)->(2*5, 6, 100/5)=(10, 6, 20)
        values的形状会变形为：(2, 6, 100)->(2*5, 6, 100/5)=(10, 6, 20)
        '''
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)
        #print('q,k,v的各自形状：', queries.shape, keys.shape, values.shape)
        
        if valid_lens is not None:
            # 在轴0，将第一项（标量或者矢量）复制num_heads次，
            # 然后如此复制第二项
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0) #本例中应该是长度为10的向量，5个2和5个3
            #print('复制后的valid_lens形状：', valid_lens.shape)

        # output的形状:(batch_size*num_heads，查询的个数，num_hiddens/num_heads)
        ## 此处为(2*5, 4, 100/5)
        output = self.attention(queries, keys, values, valid_lens)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)

In [20]:
# 如何间隔复制的对象是向量
h = torch.tensor([2,3])
h0 = torch.repeat_interleave(h, 5, dim=0)
h0.shape, h0

(torch.Size([10]), tensor([2, 2, 2, 2, 2, 3, 3, 3, 3, 3]))

- 为了能够使多个头并行计算，
上面的`MultiHeadAttention`类将使用下面定义的两个转置函数。
- 具体来说，`transpose_output`函数反转了`transpose_qkv`函数的操作。


In [21]:
#@save
'''
X形状：(2,4,100)或(2,6,100)
num_heads:5
'''
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，
    # num_hiddens/num_heads)
    '''
    本步骤的变形
    queries：(2,4,100)->(2,4,5,20)
    keys,values: (2,6,100)->(2,6,5,20)
    '''
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)

    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    '''
    本步骤的变形
    queries：(2,4,5,20)->(2,5,4,20)
    keys,values: (2,6,5,20)->(2,5,6,20)
    '''
    X = X.permute(0, 2, 1, 3)

    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    '''
    本步骤的变形
    queries：(2,5,4,20)->(10,4,20)
    keys,values: (2,5,6,20)->(10,6,20)
    
    可不可以这样解释：其实就是将注意力头个数的维度变形到了第1维，
    便于后面进行张量计算。计算完之后，再通过transpose_output
    变回来就是多头注意力的计算结果了。
    '''
    
    return X.reshape(-1, X.shape[2], X.shape[3])


#@save
'''
以queries为例，多头注意力的计算值(非最终值)X是(10,4,20),然后要变回查询queries的形状(2,4,100)。
'''
def transpose_output(X, num_heads):
    
    """逆转transpose_qkv函数的操作"""
    '''
    (10,4,20)->(2,5,4,20)，此处是将注意力头数从第1维中分离出来成为单独的第2维。
    '''
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    
    '''
    (2,5,4,20)->(2,4,5,20),将注意力头数重排到第3维，便于后面合并多头注意力的输出值。
    '''
    X = X.permute(0, 2, 1, 3)
    
    '''
    (2,4,5,20)->(2,4,100),合并第3,4维，作为多头注意力的值进行输出。
    可以这样理解，在4个查询中，每个都有100个注意力，然后又分为5个头，每个头20个注意力值。
    这5个分别捕获序列内各种范围的依赖关系，比如长距离依赖和短距离依赖 。
    '''
    return X.reshape(X.shape[0], X.shape[1], -1)

- 下面使用键和值相同的小例子来测试编写的`MultiHeadAttention`类。
- 多头注意力输出的形状是（`batch_size`，`num_queries`，`num_hiddens`）。


In [22]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                               num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [23]:
batch_size, num_queries = 2, 4
num_kvpairs, valid_lens =  6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
## queries:X, keys:Y, values:Y
attention(X, Y, Y, valid_lens).shape

torch.Size([2, 4, 100])

In [24]:
valid_lens.shape, X.shape, Y.shape

(torch.Size([2]), torch.Size([2, 4, 100]), torch.Size([2, 6, 100]))

## 小结

* 多头注意力融合了来自于多个注意力池化的不同知识，这些知识的不同来源于相同的查询、键和值的不同的子空间表示。
* 基于适当的张量操作，可以实现多头注意力的并行计算。

----
- **说明：多头注意力形成多头的3种方式**
  - **线性投影分头法**（最常用）：
```python
# 假设输入 x 的形状是 [batch_size, seq_len, d_model]
# num_heads 是头数
# d_k 是每个头的维度

# 通过线性变换得到 Q、K、V
Q = linear_q(x)  # [batch_size, seq_len, d_model]
K = linear_k(x)  # [batch_size, seq_len, d_model]
V = linear_v(x)  # [batch_size, seq_len, d_model]

# 重塑为多头形式
Q = Q.view(batch_size, seq_len, num_heads, d_k).transpose(1, 2)
K = K.view(batch_size, seq_len, num_heads, d_k).transpose(1, 2)
V = V.view(batch_size, seq_len, num_heads, d_k).transpose(1, 2)
# 最终形状：[batch_size, num_heads, seq_len, d_k]
```

  - **直接分割法**：
```python
# 直接将输入特征维度分成多头
x = x.view(batch_size, seq_len, num_heads, -1).transpose(1, 2)
```

  - **卷积分头法**：
```python
# 使用分组卷积来实现多头
conv = nn.Conv1d(in_channels, out_channels, 
                 kernel_size=1, groups=num_heads)
```

  - **区别和特点**：
    - 线性投影分头法：
      - 最常用的方法
      - 每个头可以学习不同的特征表示
      - 通过可学习的参数进行特征变换
      - 计算开销较大但效果更好
    - 直接分割法：
      - 实现简单，计算开销小
      - 没有额外的参数学习
      - 可能会限制特征学习能力
      - 各个头之间的特征相关性可能较强
    - 卷积分头法：
      - 结合了卷积的特点
      - 可以捕获局部特征
      - 参数量适中
      - 适合某些特定任务
     
----